<a id="top"></a>

# Creating a Color-Magnitude Diagram for Messier 13
***

## Learning Goals 
By the end of this tutorial, you will:

- Understand how to use `astroquery.mast` to download the SDSS Legacy Imaging Survey Data from the MAST archive.
- Plot a Color-Magnitude Diagram of Messier 13
- Calculate the distance to Messier 13 using the Gaia catalog
- Approximate the age of Messier 13 using stellar isochrones

## Table of Contents
* [Introduction](#introduction)
* [Imports](#imports)
* [Accessing SDSS Legacy Imaging Survey Data from MAST](#accessing-sdss-legacy-imaging-survey-data-from-mast)
    * [Querying SDSS Legacy Imaging Survey](#querying-sdss-legacy-imaging-survey) 
    * [Sampling SDSS Legacy Imaging Survey for M13](#sampling-sdss-legacy-imaging-survey-for-m13)
    * [Downloading Data Products](#downloading-sdss-legacy-imaging-survey-data-products)
    * [Creating a Color-Magnitude Diagram](#creating-a-color-magnitude-diagram-of-m13)
* [Finding the Distance to M13 Using Gaia](#finding-the-distance-to-m13-using-data-from-gaia)
* [Determining the Age of M13 Using Stellar Isochrone Matching](#determining-the-age-of-m13-using-stellar-isochrone-matching)
* [End of Tutorial](#end-of-tutorial)
* [Exercises](#exercises)
* [Exercise Solutions](#exercise-solutions)
* [Additional Resources](#addtional-resources)
* [Citations](#citations)
* [About this Notebook](#about-this-notebook)

## Introduction 
The Hertzsprung-Russell (HR) Diagram was independently discovered by Ejnar Hertzsprung and Henry Norris Russell in the early 20th century. This diagram effectively plots a star's theoretical luminosity against its temperature.
By examining the position of stars on the HR diagram, we can infer stellar properties such as age, metallicity, or mass. An HR Diagram is also useful for tracking the life cycle of stars. Stars in their hydrogen-core burning phases evolve leftwards on the HR diagram. Eventually, they leave the main sequence to start their red giant phases and move upwards as they begin burning helium in their cores [[1]](https://astronomy.swin.edu.au/cosmos/h/hertzsprung-russell+diagram).

An observational variation of the HR Diagram is the *Color-Magnitude Diagram* (CMD), which uses color (a proxy for temperature) and magnitude (a proxy for luminosity). We can use photometric data from a star cluster to create a CMD, and then estimate the age by comparing the main sequence tracks and red giant branch to theoretical isochrones — curves that represent stars of the same age but with different masses, mapping their expected positions at various evolutionary stages. For example, an older cluster will have more stars on the red giant branch whereas a younger cluster will have more stars on the main sequence. The transition between the two evolutionary stages, called the *Main Sequence Turn-Off*, is the key to determining the age of a stellar cluster [[2]](https://astronomyonline.org/Astrophotography/CMDDiagram.asp)

<figure style="text-align: center;">
  <img src="https://www.researchgate.net/profile/Lawrence-Krauss/publication/10964320/figure/fig2/AS:601699240644635@1520467686671/A-schematic-color-magnitude-diagram-for-a-typical-globular-cluster-33-showing-the.png" alt="Color Magnitude Diagram for Globular Clusters" width="400px" height="450px"/>
  <figcaption>
  Color Magnitude Diagram for Global Clusters from L. Krauss and B. Chaboyer
  <a href="https://www.science.org/doi/10.1126/science.1075631" target="_blank">[3].</a>
  </figcaption>
</figure>

Messier 13 (M13) is a globular cluster located in the constellation Hercules [[4]](https://science.nasa.gov/mission/hubble/science/explore-the-night-sky/hubble-messier-catalog/messier-13/). M13 is estimated to contain over 100,000 stars with a total metallicity ([Fe/H]) of approximately -1.33 dex [[5]](https://academic.oup.com/mnras/article/404/3/1203/1049211?login=true#92596831). This is consistent with the ancient stellar populations found in other globular clusters [[6]](https://science.nasa.gov/universe/star-clusters-inside-the-universes-stellar-collections/). 

In this notebook, we'll estimate the age of M13 using photometric data from the Sloan Digital Sky Survey's (SDSS) Legacy Imaging Survey and stellar isochrones from the PARSEC database [[7]](https://www.aanda.org/articles/aa/full_html/2022/09/aa44166-22/aa44166-22.html). By plotting isochrone tracks for a range of ages and comparing them to the observed stellar distribution of M13, we can identify the isochrone that best fits the cluster's data and thereby estimate its age.


## Imports 
The main packages and their use-cases in this tutorial are as follows:
- *numpy* to handle array functions
- *matplotlib.pyplot* for plotting data
- *matplotlib.image* for displaying .jpg images
- *astroquery.mast.Observations* to access SDSS data from MAST
- *astroquery.mast.Catalogs* to access data Gaia data from MAST
- *pandas* to handle large data tables
- *astropy.io.fits* for accessing FITS files

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from astroquery.mast import Observations
from astroquery.mast import Catalogs
import pandas as pd
import astropy.io.fits as fits

If you're not sure if you have the required versions of packages installed on your device, you can run the following cell:

In [ ]:
with open("requirements.txt") as f:
    print(f"Required packages for this notebook:\n{f.read()}")

To ensure these requirements are installed, you can run the following command in the terminal:
```bash
pip install -r requirements.txt
```

*** 
## Accessing SDSS Legacy Imaging Survey Data from MAST 

The [Mikulski Archive for Space Telescopes (MAST)](https://archive.stsci.edu/) hosts a large array of data from several telescope missions. In this tutorial, we will be specifically focusing on data from the Sloan Digital Sky Survey's (SDSS) Imaging Survey. 

The [SDSS Legacy Imaging Survey](https://www.sdss.org/dr18/imaging/) was the first SDSS survey completed! Beginning in 1998, the SDSS Legacy Imaging Survey captured around 35,000 square degrees of images until 2009, which covers about one-third of the whole sky. To do this, the survey took millions of 10 by 13 arcminute pictures (called "fields"). Not only does the SDSS Legacy Imaging Survey data at MAST provide users with preview images in .jpg format, but it also has photometric data in all five broad band SDSS filters (u, g, r, i, and z). For more information about the data and products from the SDSS Legacy Imaging Survey, you can check out the [SDSS Imaging Archive Manual](https://outerspace.stsci.edu/display/SDSS/Legacy+Imaging). 

### Querying SDSS Legacy Imaging Survey

You can query data from MAST using the [website portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html). You can also query MAST data in Python using the package `astroquery.mast`! 

We can query all of the SDSS Legacy Imaging data using `Observations.query_criteria` with `provenance_name = "SDSS Legacy Imaging"`. Since there is a large amount of data in the SDSS Legacy Imaging archive, we can display the first 10 results using the `pagesize` parameter as well as the `page` parameter.

In [ ]:
# Querying SDSS Legacy Imaging
imaging_data = Observations.query_criteria(
    provenance_name="SDSS Legacy Imaging", pagesize=10, page=1
)

# Display first 10 entries
imaging_data

The table above provides some basic information for each object:

- `instrument_name`: `SDSS Camera` indicates that the imaging data were collected using the SDSS camera. 
- `filters`: Indicates the SDSS filters used in the survey (u, g, r, i, and z).
- `wavelength_region`: Indicates the region of the electromagnetic spectrum observed. This should be `OPTICAL`, since the SDSS Legacy Imaging Survey observed in the optical wavelength range.
- `target_classification`: `FIELD` indicates the survey captured images of the sky.
- `obs_id`: Observation ID associated with the image. 
- `s_ra` and `s_dec`: Right ascension and declination.
- `dataproduct_type`: This should be `image` since we're focusing on an imaging survey.
- `t_min` and `t_max`: The modified Julian dates indicating the start and end times of the exposures.
- `em_min` and `em_max`: The minimum and maximum wavelengths observed by the survey. For the SDSS Legacy Imaging survey, this range is approximately 304.8 - 1083.3 nanometers (optical).

### Sampling SDSS Legacy Imaging Survey for M13

In this tutorial, we will be gathering g-band and r-band magnitudes of stars in M13 measured by the SDSS Legacy Imaging survey by querying and downloading data through MAST. We'll plot these on a CMD where the apparent r-band magnitude is on the y-axis, and color (g-band minus r-band) is on the x-axis. We'll later plot this CMD with a CMD of isochrones from the [PARSEC database](https://stev.oapd.inaf.it/cgi-bin/cmd).

For the purpose of this tutorial, we will only focus on querying by object name. We can query the SDSS Legacy Imaging Survey for M13 data using `objectname="M13"`.

In [ ]:
# Querying SDSS Legacy Imaging for M13
obs_table = Observations.query_criteria(
    objectname="M13", provenance_name="SDSS Legacy Imaging"
)

# Displaying the table
obs_table

From the query results, we can see that there are 17 SDSS fields that cover M13! The distance column shows how far each field is from the center of the cluster in arcseconds. 

### Downloading SDSS Legacy Imaging Survey Data Products

For each image, there are 7 available products: The preview .jpg image, the FITS files in each respective SDSS filter (u, g, r, i, and z), and the full FITS file with all five filters combined. The full FITS file is listed as `SDSS Imaging Catalogs` for the `productSubGroupDescription` parameter. 

For more information about products available at MAST, you can check out the [SDSS Legacy Imaging Survey Documentation](https://outerspace.stsci.edu/display/SDSS/Legacy+Imaging+Data+Products). 

In [ ]:
# Viewing available products
products = Observations.get_product_list(obs_table)
products

We can examine the preview image of the field of stars we'll be creating a CMD with! We'll download the preview .jpg file to your directory using the parameter `flat=True`. 

In [ ]:
# Selecting .jpg image
img_products = Observations.filter_products(
    products,
    # Select .jpg file for photoObj
    productFilename="frame-irg-003226-5-0126.jpg",
)

In [ ]:
# Downloading preview images
Observations.download_products(img_products, flat=True, verbose=False)

Now we can preview the image! 

In [ ]:
img = mpimg.imread("frame-irg-003226-5-0126.jpg")
plt.figure(figsize=(10, 8))
imgplot = plt.imshow(img)
plt.axis("off")
plt.title("Preview Image of M13")
plt.show()

While the field isn't directly capturing the center of cluster, it will contain enough cluster stars to create a CMD with! We'll now download the full FITS files and select `photoObj-003226-5-0126.fits` due to its CMD congruency. 

Each photoObj FITS file follows the following naming scheme: `photoObj-{RUN}-{CAMCOL}-{FIELD}` where `RUN` is the zero-padded, six-digit identification number for that photometric run, `CAMCOL` is the camera column identifying the scanline within the run, and `FIELD` is field number for that observation. The photoObj file contains the fully calibrated FITS files from the [SDSS Imaging Pipeline](https://www.sdss4.org/dr17/imaging/pipeline/) and other respective measurements such as photometry in all five SDSS filters (u, g, r, i, and z), source classification, and quality flags. There is one photoObj file for each run-camcol-field combination. 

In [ ]:
# Masking the data to extract only the full FITS file
new_products = Observations.filter_products(
    products,
    # Specify FITS files only
    productGroupDescription="Minimum Recommended Products",
    # Select photoObj files (ugriz)
    productSubGroupDescription="SDSS Imaging Catalogs",
    # Select photoObj-003226-5-0126.fits
    productFilename="photoObj-003226-5-0126.fits",
)

# Downloading photoObj-003226-5-0126.fits
manifest = Observations.download_products(new_products, verbose=False)

We can now open the file and examine it. For this tutorial, we will focus on the g-band and r-band magnitudes provided in the first file extension. 

The various file extensions are broken down on the [SDSS website](https://data.sdss.org/datamodel/files/BOSS_PHOTOOBJ/RERUN/RUN/CAMCOL/photoObj.html):
- HDU 0: The primary header information.
- HDU 1: The photoObj table. This contains information such as magnitudes, coordinates, and flux.

In [ ]:
# Opening file
star_array = fits.open(manifest["Local Path"][0])

# Display header info
star_array.info()

### Creating a Color-Magnitude Diagram of M13

To obtain magnitudes from the FITS file, we'll be using `PSFMAG` from the first file extension. Within `PSFMAG` there are 5 total columns, each one corresponding to the apparent magnitudes in the u, g, r, i, or z filters. We'll extract the g-band magnitudes from the second column and the r-band magnitudes from the third column.

In [ ]:
# Sending the PSFMAG data to a numpy array
magnitudes = star_array[1].data["PSFMAG"]
magnitudes = np.array(magnitudes)

# Creating arrays for g-band and r-band magnitudes
g_band = magnitudes[:, 1]
r_band = magnitudes[:, 2]

# Calculating color (g - r)
color_data1 = g_band - r_band

To create a CMD, we simply need to plot the r-band magnitudes against the color (g-r) for these stars!

In [ ]:
# Plotting the CMD
plt.figure(figsize=(8, 6))
plt.scatter(color_data1, r_band, s=3, color="black", label="M13 data")
plt.ylim(10, 30)
plt.gca().invert_yaxis()
plt.xlim(-2, 2)
plt.title("CMD of M13")
plt.ylabel("Apparent Magnitude (r-band)")
plt.xlabel("Color (g - r)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

*** 
## Finding the Distance to M13 Using Data from Gaia

Gaia was launched by the European Space Agency (ESA) in December 2013. Orbiting at L2, its main purpose is to observe billions of stars to create a 3D map of our galaxy. Gaia provides data such as astrometry, photometry, and radial velocities For more information about Gaia, visit the [ESA website](https://www.esa.int/Science_Exploration/Space_Science/Gaia). 

We'll query Gaia for M13 data using a search radius of 0.375 degrees. This narrows our search so that we focus on the cluster stars rather than foreground stars, allowing us to get a more accurate distance.

The catalog search should take around a minute or so. 

In [ ]:
# querying Gaia
coordinates = "16h41m41.24s 36d27m35.5s"  # Coordinates of M13
catalog_data = Catalogs.query_region(
    coordinates, catalog="GaiaDR3", radius=0.375
)

Gaia provides parallax measurements for a wide variety of astronomical objects. Stellar parallax is the apparent shift in position of a nearby star caused by Earth's motion around the Sun.

<figure style="text-align: center;">
  <img src="https://www.astronomy.ohio-state.edu/pogge.1/Ast162/Unit1/Images/parallax_sm.png" alt="Stellar Parallax" width="450px" height="320px"/>
  <figcaption>
  Stellar Parallax Diagram by R. Pogge
  <a href="https://www.astronomy.ohio-state.edu/pogge.1/Ast162/Unit1/distances.html" target="_blank">[8].</a>
  </figcaption>
</figure>

We can determine the distance to these objects from the parallax angle using small-angle approximation:

$$
d = \frac{1}{p}
$$

where $d$ is distance in parsecs and $p$ is parallax in arcseconds. 

In [ ]:
# Extracting parallax measurements from Gaia
parallax = catalog_data["parallax"]  # in milliarcseconds
distance_array = 1 / parallax  # distance array from parallax (kiloparsecs)
ra = catalog_data["ra"]  # right ascension
dec = catalog_data["dec"]  # declination

# Sorting data
sorted_indices = np.argsort(distance_array)
ra_sorted = ra[sorted_indices]
dec_sorted = dec[sorted_indices]
distance_sorted = distance_array[sorted_indices]

# Creating a histogram of the parallax measurements and RA/DEC map
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# Histogram
ax[0].hist(parallax, bins=100, color="gray")
ax[0].axvline(
    np.nanmedian(parallax),
    color="red",
    label=f"Median Parallax \n {np.nanmedian(parallax):.3f} milliarcsec",
)
ax[0].set_xlim(-5, 5)
ax[0].set_xticks(np.arange(-5, 5, 1))
ax[0].set_xlabel("Parallax (milliarcseconds)")
ax[0].set_ylabel("Frequency")
ax[0].set_title("Histogram of M13 Parallax Measurements from Gaia")
ax[0].legend()

# RA/DEC map
plot = ax[1].scatter(
    ra_sorted,
    dec_sorted,
    s=2,
    c=distance_sorted,
    cmap="viridis",
    vmin=0,
    vmax=8,
)
cbar = fig.colorbar(plot, ax=ax[1])
cbar.set_label("Distance (kpc)")
ax[1].set_xlabel("RA (degrees)")
ax[1].set_ylabel("DEC (degrees)")
ax[1].set_title("Position of Parallax Measurements Colored by Distance")

plt.show()

The histogram above shows the distribution of the various parallax measurements Gaia made for M13. We'll take the median of these measurements to use as a general distance for the cluster. 

In [ ]:
# Finding the median parallax of the data
cluster_median = np.nanmedian(parallax)

# Calculating distance using stellar parallax formula
distance = 1.0 / cluster_median

print(f"Median Parallax: {cluster_median} milliarcseconds")
print(f"Distance to M13: {distance:.4f} kpc")

Therefore, according to Gaia data, the distance to M13 is approximately 7.6951 kpc. We can now use this to convert the isochrones to apparent magnitudes in the next steps. 

***
## Determining the Age of M13 Using Stellar Isochrone Matching

We can determine the approximate age of M13 by plotting its CMD with isochrones of varying ages, and match the best fit isochrone track to the M13 data. 

The isochrones are located in the .txt file `isochrones.txt`. These were the result of generating isochrones from the [PARSEC database](https://stev.oapd.inaf.it/cgi-bin/cmd) with a metallicity of -1.33 dex and ages from 2 billion years old to 14 billion years old in step sizes of 1 billion years. The other settings were left as default. 

Since the SDSS Legacy Survey observed apparent magnitudes, we'll need to convert the absolute magnitudes from the isochrones to apparent magnitudes. We'll use the distance modulus formula since we know the approximate distance to M13 [[9]](https://astronomy.swin.edu.au/cosmos/d/Distance+Modulusx):
$$
m-M=5log_{10}(\frac{d}{10})
$$
where $m$ is apparent magnitude, $M$ is absolute magnitude, and $d$ is the distance to the object in parsecs. 

In [ ]:
# Distance to M13
d = distance * 10**3  # parsecs


# Converting absolute magnitudes from isochrones to apparent
def app_mag(M):
    return M + 5 * np.log10(d / 10)

In [ ]:
# Importing isochrone .txt file
parsec_data = pd.read_csv(
    "isochrones.txt", sep=r"\s+", comment="#", header=None
)

# Converting the data file to an array
parsec_data = np.array(parsec_data)

In [ ]:
# The isochrone data file is broken up into age bins
log_age = [
    9.30103,
    9.47712,
    9.60206,
    9.69897,
    9.77815,
    9.84510,
    9.90309,
    9.95424,
    10.00000,
    10.04139,
    10.07918,
    10.11394,
    10.14613,
]

# Setting up a dictionary
age_data = {}

# Looping through .txt file to separate by age
for i, age in enumerate(log_age, start=1):
    age_data[f"age{i}"] = parsec_data[parsec_data[:, 2] == age]

# Extracting g and r magnitudes, applying distance modulus formula
# Setting up arrays
r_data = []
g_data = []
color_data2 = []

# Sending g and r magnitudes and color to an array
for i in range(1, len(log_age) + 1):
    data = age_data[f"age{i}"]
    g_mag = app_mag(data[:, 29])
    r_mag = app_mag(data[:, 30])
    color = g_mag - r_mag

    r_data.append(r_mag)
    g_data.append(g_mag)
    color_data2.append(color)

# Plotting the isochrones
color = [
    "brown",
    "pink",
    "magenta",
    "purple",
    "navy",
    "darkturquoise",
    "teal",
    "olive",
    "green",
    "gold",
    "orange",
    "lightcoral",
    "red",
]
fig, ax = plt.subplots(figsize=(10, 8))
for i, j, k, r in zip(color_data2, r_data, log_age, color):
    ax.plot(
        i,
        j,
        alpha=0.5,
        label=f"{(10**k) / 1e9:.0f} billion years old",
        color=r,
    )

# Plotting the data
ax.scatter(
    color_data1,
    r_band,
    marker="x",
    linewidth=0.9,
    s=20,
    color="black",
    label="SDSS data of M13",
)
ax.set_ylabel("Apparent Magnitude (r-band)")
ax.set_xlabel("Color (g - r)")
ax.set_title("Color - Magnitude Diagram of M13 with Isochrones")
ax.set_ylim(10, 30)
ax.set_xlim(-2, 2)
ax.invert_yaxis()
ax.legend(fontsize=10)
ax.grid(True)
fig.tight_layout()

While the isochrones extend beyond 27.5 r-band magnitude, the M13 data doesn't quite reach this far due to the limiting magnitude of the SDSS camera. 

We'll create a zoomed-in plot to further study the terminal age main sequence (TAMS) and main sequence turnoff phases. 

In [ ]:
# Replotting with different x and y limits
ax.set_xlim(0, 0.9)
ax.set_ylim(16, 20)
ax.invert_yaxis()
ax.legend(fontsize=8)

# Adding annotation for main sequence turnoff
circle = plt.Circle(
    (0.35, 17.75),
    0.25,
    color="red",
    fill=False,
    linewidth=2,
    label="Main Sequence Turnoff",
)
ax.add_patch(circle)
ax.annotate(
    "Main Sequence Turnoff",
    xy=(0.6, 17.75),
    xytext=(0.6 + 0.05, 17.75 - 0.2),
    arrowprops=dict(arrowstyle="->", color="red"),
    fontsize=12,
    color="red",
)

fig

From examining the data and isochrones within the main sequence turnoff circle, it appears the data follow the orange track, indicating the cluster is approximately **12 billion years old**. This isn't far from the 11.65 billion year old estimate made by D. Forbes and T. Bridges in *Accreted Versus in situ Milky Way Globular Clusters* [[5]](https://academic.oup.com/mnras/article/404/3/1203/1049211?login=true#92596831).

***
## End of Tutorial

Congratulations, you now know how to determine the age of a star cluster from isochrone matching using the SDSS Legacy Imaging Survey! 

***
## Exercises
Now it's your turn to determine the age of **Messier 3**! M3 has a metallicity of -1.34 dex, which is nearly identical to M13, so you can use the same isochrone file `isochrones.txt` [[5]](https://academic.oup.com/mnras/article/404/3/1203/1049211?login=true#92596831). 

There are six different fields available for M3, you can choose whichever one visually has the best looking CMD when plotted, so answers can vary slightly.  

In [ ]:
# Query SDSS Legacy Imaging Survey for M3 data

In [ ]:
# View available products

In [ ]:
# Download products

In [ ]:
# Plot CMD of M3

In [ ]:
# Find distance to M3 using Gaia

In [ ]:
# Plot CMD with isochrones

***
## Exercise Solutions

In [ ]:
# Query SDSS Legacy Imaging Survey for M3 data

obs_table2 = Observations.query_criteria(
    objectname="M3", provenance_name="SDSS Legacy Imaging"
)

# Displaying the table
obs_table2

In [ ]:
# Download products
products2 = Observations.get_product_list(obs_table2)

# Download products
new_products2 = Observations.filter_products(
    products2,
    # Specify FITS files only
    productGroupDescription="Minimum Recommended Products",
    # Select photoObj files (ugriz)
    productSubGroupDescription="SDSS Imaging Catalogs",
    # Select photoObj-003226-5-0126.fits
    productFilename="photoObj-004649-4-0145.fits",
)

# Downloading photoObj-003226-5-0126.fits
manifest2 = Observations.download_products(new_products2)

In [ ]:
# Plot CMD of M3

# Opening file
star_array2 = fits.open(manifest2["Local Path"][0])

# Sending the PSFMAG data to a numpy array
magnitudes2 = star_array2[1].data["PSFMAG"]
magnitudes2 = np.array(magnitudes2)

# Creating arrays for g-band and r-band magnitudes
g_band2 = magnitudes2[:, 1]
r_band2 = magnitudes2[:, 2]

# Calculating color (g - r)
color_data3 = g_band2 - r_band2

# Plotting the CMD
plt.figure(figsize=(8, 6))
plt.scatter(color_data3, r_band2, s=3, color="black", label="M3 Data")
plt.ylim(10, 30)
plt.gca().invert_yaxis()
plt.xlim(-2, 2)
plt.title("CMD of M3")
plt.ylabel("Apparent Magnitude (r-band)")
plt.xlabel("Color (g - r)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Find distance to M3 using Gaia

# querying Gaia
coordinates2 = "13h42m11.80s 28d22m31.69s"  # Coordinates of M3

catalog_data2 = Catalogs.query_region(
    coordinates2, catalog="GaiaDR3", radius=0.35
)

# Extracting parallax measurements from Gaia
parallax2 = catalog_data2["parallax"]  # in milliarcseconds
distance_array2 = 1 / parallax2  # distance array from parallax (kiloparsecs)
ra2 = catalog_data2["ra"]  # right ascension
dec2 = catalog_data2["dec"]  # declination

# Sorting data
sorted_indices2 = np.argsort(distance_array2)
ra_sorted2 = ra2[sorted_indices2]
dec_sorted2 = dec2[sorted_indices2]
distance_sorted2 = distance_array2[sorted_indices2]

# Creating a histogram of the parallax measurements and RA/DEC map
fig2, ax2 = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

# Histogram
ax2[0].hist(parallax2, bins=100, color="gray")
ax2[0].axvline(
    np.nanmedian(parallax2),
    color="red",
    label=f"Median Parallax \n {np.nanmedian(parallax2):.3f} milliarcsec",
)
ax2[0].set_xlim(-5, 5)
ax2[0].set_xticks(np.arange(-5, 5, 1))
ax2[0].set_xlabel("Parallax (milliarcseconds)")
ax2[0].set_ylabel("Frequency")
ax2[0].set_title("Histogram of M3 Parallax Measurements from Gaia")
ax2[0].legend()

# RA/DEC map
plot2 = ax2[1].scatter(
    ra_sorted2,
    dec_sorted2,
    s=2,
    c=distance_sorted2,
    cmap="viridis",
    vmin=0,
    vmax=8,
)
cbar2 = fig.colorbar(plot2, ax=ax2[1])
cbar2.set_label("Distance (kpc)")
ax2[1].set_xlabel("RA (degrees)")
ax2[1].set_ylabel("DEC (degrees)")
ax2[1].set_title("Position of Parallax Measurements Colored by Distance")

plt.show()

# Finding the median parallax of the data
cluster_median2 = np.nanmedian(parallax2)

# Calculating distance using stellar parallax formula
distance2 = 1.0 / cluster_median2

print(f"Median Parallax: {cluster_median2} milliarcseconds")
print(f"Distance to M3: {distance2:.4f} kpc")

# Distance to M3
d2 = distance2 * 10**3  # parsecs

In [ ]:
# Plot CMD with isochrones


def app_mag2(M):
    return M + 5 * np.log10(d2 / 10)


r_data2 = []
g_data2 = []
color_data4 = []

for i in range(1, len(log_age) + 1):
    data = age_data[f"age{i}"]
    g_mag = app_mag2(data[:, 29])
    r_mag = app_mag2(data[:, 30])
    color2 = g_mag - r_mag

    r_data2.append(r_mag)
    g_data2.append(g_mag)
    color_data4.append(color2)

fig, ax = plt.subplots(figsize=(10, 8))
for i, j, k, r in zip(color_data4, r_data2, log_age, color):
    ax.plot(
        i,
        j,
        alpha=0.5,
        label=f"{(10**k) / 1e9:.0f} billion years old",
        color=r,
    )

# Plotting the data
ax.scatter(
    color_data3,
    r_band2,
    marker="x",
    linewidth=0.9,
    s=20,
    color="black",
    label="SDSS data of M3",
)
ax.set_ylabel("Apparent Magnitude (r-band)")
ax.set_xlabel("Color (g - r)")
ax.set_title("Color - Magnitude Diagram of M3 with Isochrones")
ax.set_ylim(10, 30)
ax.set_xlim(-2, 2)
ax.invert_yaxis()
ax.legend(fontsize=10)
ax.grid(True)
fig.tight_layout()

In [ ]:
# Replotting with different x and y limits
ax.set_xlim(0, 0.9)
ax.set_ylim(16, 20)
ax.invert_yaxis()
ax.legend(fontsize=8)

# Adding annotation for main sequence turnoff
circle = plt.Circle(
    (0.35, 18.25),
    0.25,
    color="red",
    fill=False,
    linewidth=2,
    label="Main Sequence Turnoff",
)
ax.add_patch(circle)
ax.annotate(
    "Main Sequence Turnoff",
    xy=(0.6, 18.25),
    xytext=(0.6 + 0.05, 17.75 - 0.2),
    arrowprops=dict(arrowstyle="->", color="red"),
    fontsize=12,
    color="red",
)

fig

**Results:**
The data for M3 appear to fit between the yellow and orange lines. Therefore, M3 is between 11-12 billion years old! 

This matches the [age estimates](https://www.kopernik.org/images/archive/m3.htm) of approximately 8-11.4 billion years.

***
## Addtional Resources

Additional resources are linked below:
- [SDSS Legacy Archive at MAST](https://archive.stsci.edu/missions-and-data/sdss)
- [SDSS Legacy Archive at MAST User Manual](https://outerspace.stsci.edu/display/SDSS/The+SDSS+Legacy+Archive+at+MAST)
- [SDSS Legacy Imaging Archive User Manual](https://outerspace.stsci.edu/display/SDSS/Legacy+Imaging)
- [SDSS Imaging Documentation](https://www.sdss.org/dr18/imaging/)
- [SDSS Voyages HR Diagram Tutorial](https://voyages.sdss.org/expeditions/expedition-to-the-milky-way/star-clusters/hr-diagrams/)
- [astroquery.mast User Manual](https://astroquery.readthedocs.io/en/latest/mast/mast.html)
- [MAST API](https://mast.stsci.edu/api/v0/index.html)

## Citations

If you use data from MAST for published research, please see the following links for information on which citations to include in your paper:

* [Citing SDSS](https://sdss.org/collaboration/citing-sdss/)
* [Citing MAST](https://archive.stsci.edu/publishing/mission-acknowledgements)
* [Citing astropy](https://www.astropy.org/acknowledging.html)

## About this Notebook

**Author(s):** Natalie Haugen (nhaugen@terpmail.umd.edu) and Julie Imig (jimig@stsci.edu) <br>
**Keyword(s):** Tutorial, SDSS, SDSS Legacy Imaging Survey, stars, HR diagram, CMD, imaging <br>
**First published:** July 2025 <br>
**Last updated:** July 2025 <br>

***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/style-guides/master/guides/images/stsci-logo.png" alt="Space Telescope Logo" width="200px"/>